In [1]:
from ollama import chat
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import pandas as pd

**Importing the model and the dataset**

In [2]:
model = OllamaLLM(model="deepseek-r1:1.5b")

In [3]:
aggregatedProjectsDF = pd.read_csv('projectsAggWResults.csv',sep=",")
aggregatedProjectsDF = aggregatedProjectsDF.drop('Unnamed: 0',axis=1)
electionDetails = pd.read_csv('projectDetails.csv',sep=";")

Testing the similarity analysis

In [4]:
test_template_en = """
    Given this short project description: {target_project}

    Compare the description given with the following texts as rank them in order of similarity
    to the given descriptions and organize them from the most similar to the least similar:
    1-{desc_1},
    2-{desc_2},
    3-{desc_3}

"""

In [5]:
prompt_en = ChatPromptTemplate.from_template(test_template_en)
chain = prompt_en | model


In [6]:
test_input = chain.invoke({
    "target_project":"Il est important de boire suffisamment d’eau chaque jour pour rester en bonne santé.",
    "desc_1":"Une alimentation équilibrée contribue aussi à améliorer l’énergie et la concentration.",
    "desc_2":"Le volcan est entré en éruption soudainement, couvrant le ciel de cendres.",
    "desc_3":"S’hydrater régulièrement aide le corps à bien fonctionner et prévient la fatigue."
})

print(test_input)

<think>
Alright, so I need to figure out how to rank these three texts in order of their similarity to the given short project description about sufficient water each day. Let me break this down step by step.

First, let's understand what the user is asking for. They provided a specific sentence: "Il est important de boire suffisamment d’eau chaque jour pour rester en bonne santé." This translates to "It is essential to drink enough water every day to stay in good health."

Then, they gave three other texts and asked me to compare them with this description on a scale of similarity. They also want the ranked order from most similar to least similar.

So, I need to analyze each of the three texts one by one and see how closely they match the project's description. Let's look at each text:

1. "Une alimentation équilibrée contribue aussi à améliorer l’énergie et la concentration.,"
2. "Le volcan est entré en éruption soudainely, couvrant le ciel de cendres.,"
3. "S’hydrater régulièrement

As we can see, the model can do the comparisons between short texts, it takes some time but it works, now we'll need to see how does it works on a context more similar to our context

In [7]:
print(aggregatedProjectsDF.columns)

Index(['project_id', 'project_name', 'description', 'category', 'cost',
       'district', 'votes', 'title_length', 'description_length',
       'agg_district_code', 'agg_quartiers', 'src_district_code', 'approved',
       'approved_binary'],
      dtype='object')


We are going to apply this analysis district by district and check the outputted results.  
Info on how to format the output https://ollama.com/blog/structured-outputs
**Ollama documentation**  
https://github.com/ollama/ollama/blob/main/docs/api.md  
https://github.com/ollama/ollama-python  
https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-chat-completion  
https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values

In [8]:
class PairResponseFormatter(BaseModel):
    target_project: int = Field(description="ID of the project of reference that is being used to compare to other projects")
    similar_project: int = Field(description="The id of the most similar project related to the target project")

class ResponseFormatter(BaseModel):
    target_project: int = Field(description="ID of the project of reference that is being used to compare to other projects")
    ranked_similar_projects: list[int] = Field(description="The id's of similar projects ranked by their similarity from the most similar to the least similar")
    # ranked_similar_projects_score: list[float] = Field(description="the score of the similarity between projects ranked by their similarity from the most similar to the least similar")


class DetailedResponseFormatter(BaseModel):
    target_project: int = Field(description="ID of the project of reference that is being used to compare to other projects")
    ranked_similar_projects: dict[str,float] = Field(description="A dictionary where the key is the project ID and the value is the score of similarity between the projects")

In [9]:
# response = chat(
#     messages = [
#         {
#             'role': 'user',
#             'content': """
#     Given this target project description in french: 
#     target_project : {"id": 1, "description": "Développement d'une application mobile pour la gestion des tâches personnelles avec rappels et synchronisation cloud."}

#     Compare the given description to this other descriptions, formatted in a json format and give me the ID of the project that is
#     the most similar one to the target_project. Consider only similarity between descriptions

#     Other descriptions:
#     [
#         {"id": 4, "description": "Création d'une application mobile de gestion des tâches personnelles, intégrant rappels automatiques et synchronisation entre appareils."},
#         {"id": 3, "description": "Mise en place d'un système web pour la gestion de projets collaboratifs avec calendrier partagé."},
#         {"id": 2, "description": "Développement d'un jeu vidéo éducatif pour enfants en bas âge, avec des animations interactives."}
#     ]

#     Respond with the id of the target project, a list of id's of the similar projects ranked by their similarity ordered by the most similar to the least similar
#     and include in the project list even the projects that are not related at all
#     """
#         }
#     ],
#     model='deepseek-r1:1.5b',
#     format = ResponseFormatter.model_json_schema(),
# )

# test_pair = ResponseFormatter.model_validate_json(response.message.content)
# print(test_pair)

**Now that twe can use the LLM to process the similarity between projects and output that relation in a structured format using a custom prompt, we nee to find a way to compute the overlapping of projects voters**

In [10]:
def compute_voter_overlap(target_project_id:int, compared_project_id:int, election_details_dataset) -> float:
    result = {
        "overlapping_voters":[],
        "overlap_percentage_target_project":0,
        "overlap_percentage_compared_project":0
    }
    
    ##Get the voters per project
    target_project_votes = election_details_dataset[election_details_dataset['project_id']==target_project_id]
    compared_project_votes = election_details_dataset[election_details_dataset['project_id']==compared_project_id]


    ###Get a list of voters ID's that voted for that same project
    target_project_voters = target_project_votes['ID'].unique().tolist()
    compared_project_voters = compared_project_votes['ID'].unique().tolist()

    ##Calculate the overlapping percentage
    overlap_set = list(set(target_project_voters) & set(compared_project_voters))

    result["overlapping_voters"] = overlap_set
    result["overlap_percentage_target_project"]= len(overlap_set)/len(target_project_voters)
    result["overlap_percentage_compared_project"]= len(overlap_set)/len(compared_project_voters)    

    return result
    

In [11]:
compute_voter_overlap(52, 51, electionDetails)

{'overlapping_voters': [2465,
  3425,
  2533,
  4519,
  2408,
  3401,
  2139,
  555,
  3373,
  3118,
  558,
  1491,
  1749,
  1401,
  4539],
 'overlap_percentage_target_project': 0.15625,
 'overlap_percentage_compared_project': 0.38461538461538464}

In [89]:
district_projects = aggregatedProjectsDF[aggregatedProjectsDF['src_district_code'] == 5.3].copy()
district_winning_projects = district_projects[district_projects['approved_binary']==1]
district_losing_projects = district_projects[district_projects['approved_binary']==0]
print("Winning projects\n",district_winning_projects[['project_id','project_name', 'votes']].sort_values(by='votes', ascending=False))
print("Loosing projects\n", district_losing_projects[['project_id','project_name', 'votes']].sort_values(by='votes', ascending=False))

Winning projects
      project_id                                       project_name  votes
34          156  Aménager le Rond-Point des Français Libres et ...    178
128         157  Plantation d'arbres et diversification de la p...    128
149         153                Réduction de la pollution lumineuse    113
58          155              Lutter contre la pollution des mégots     63
3           149  Un jardin en mouvement le long de la voie ferr...     50
Loosing projects
      project_id                                       project_name  votes
165         151                     Aménager la place Henri Russel     98
144         150           Coup de pinceau sur le béton du quartier     56
99          154                         Nichoirs pour des mésanges     44
159         148             Plan du voisinage aux sorties de métro     24
187         152  Aire de jeux accessible à tous au jardin du Pa...     14


#### Given the objective of this study, the proof of concept of this pipeline is going to go over three cases

1. **Cloned projects** - Two projects that are very similar (similar description) - They share (ideally) no voter overlap -> Possible projects: (28,27), (51, 52), (95, 92), (151[98 votes], 156[178 votes])**
2. **The project with a very low vote count** - Compare that project with a project that was approved -> Possible projects: (176 [2 votes], 169 [189 votes]),(178 [2 votes], 182 [33 votes]), (50 [2 votes], 44 [33 votes])
3. **Two distinct projects**, one passed and the other one didn't and they overlap in voting intention (the same people that voted for project A also voted for project B but only project B was accepted) -> Possible projects: (95, 91), (103,102), (109,110), (122,125), (151, 156)**

*The pairs are the project id's*  
**This pairing fits in more than one case

In [90]:
test_project_description = district_losing_projects[district_losing_projects['project_id']==151]
# print(test_project_description)
target_project = {"id":test_project_description.iloc[0]['project_id'].item(), "description":test_project_description.iloc[0]['description']}
# print(case_1_test_project_description['project_name'])
# print(len(district15_losing_projects.index))
winning_projects = []
for index, row in district_winning_projects.iterrows():
    project = {"id":row['project_id'], "description":row['description']}
    winning_projects.append(project)

In [91]:
similarity_template_en = f"""
    You are given a target project description written in French:

target_project:
{target_project}

You are also given a list of other project descriptions in JSON format, each with an associated ID:

{winning_projects}

Your task is to compare the semantic content of the target project description to each of the other descriptions. Determine which projects are most similar based **only on the semantic similarity between the descriptions** (ignore any metadata, IDs, or structure beyond the text content).

Return a ranked list of all the project IDs, from most similar to least similar. Include all projects, even those that are not semantically related.

(Optional Example to guide behavior):

Example:
Target: {{"id": "p4", "description":"La construction d'une école primaire écologique en région rurale."}}
Other projects:
[
  {{"id": "p1", "description": "Création d'un centre médical."}},
  {{"id": "p2", "description": "Construction d'un bâtiment scolaire pour 500 élèves."}},
  {{"id": "p3", "description": "Aménagement d’un parc de loisirs."}}
]
Most similar to least similar:
p2, p1, p3
"""

In [92]:
print(similarity_template_en)


    You are given a target project description written in French:

target_project:
{'id': 151, 'description': "La place Henri Russel est aujourd'hui entièrement minérale au sol, à l'exception des arbres qui l'entourent, elle est globalement délaissée par les habitants qui ne l'investissement pas. Son usage et sa conception doivent être repensés à l'instar de l'appel à idée qui est lancé. Proposition d'aménagement la concernant : - créer une barrière végétale qui l'isole de l'avenue Crampel - créer un espace de jeu pour enfant sur la thématique des 7 grottes d'Henri RusselModalités de réalisation - Avis technique : L'amélioration de la place Henri Russell sera envisagée comme cela :- Assurer l’accessibilité de la place et de ses traversées piétonnes;\xa0- Végétalisation- Installations de davantage d’assises;\xa0- Favoriser les déplacements modes doux en implantant des racks vélos et en déplaçant la station de station Vélô Toulouse;\xa0- Conserver l’offre de stationnement;\xa0- Mettre e

In [78]:
similarity_response = chat(
    messages = [
        {
            'role': 'user',
            'content': similarity_template_en
        }
    ],
    model='deepseek-r1:1.5b',
    format = ResponseFormatter.model_json_schema(),
)

similarity_response = ResponseFormatter.model_validate_json(similarity_response.message.content)
print(similarity_response)

target_project=28 ranked_similar_projects=[25, 29, 27, 20, 24, 19, 16, 17, 18, 14, 3, 15, 11, 10, 12, 13, 21, 23, 22, 26, 27, 25, 26, 27, 28, 29, 24, 20, 19, 16, 17, 18, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


**CASE 3-Overlapping voters case**

In [57]:
case_3_district_projects = aggregatedProjectsDF[aggregatedProjectsDF['src_district_code'] == 5.3].copy()
case_3_district_winning_projects = case_3_district_projects[case_3_district_projects['approved_binary']==1]
case_3_district_losing_projects = case_3_district_projects[case_3_district_projects['approved_binary']==0]
print("CASE 3 Winning projects\n",case_3_district_winning_projects[['project_id','project_name', 'votes']].sort_values(by='votes', ascending=False))
print("CASE 3 Loosing projects\n", case_3_district_losing_projects[['project_id','project_name', 'votes']].sort_values(by='votes', ascending=False))


CASE 3 Winning projects
      project_id                                       project_name  votes
34          156  Aménager le Rond-Point des Français Libres et ...    178
128         157  Plantation d'arbres et diversification de la p...    128
149         153                Réduction de la pollution lumineuse    113
58          155              Lutter contre la pollution des mégots     63
3           149  Un jardin en mouvement le long de la voie ferr...     50
CASE 3 Loosing projects
      project_id                                       project_name  votes
165         151                     Aménager la place Henri Russel     98
144         150           Coup de pinceau sur le béton du quartier     56
99          154                         Nichoirs pour des mésanges     44
159         148             Plan du voisinage aux sorties de métro     24
187         152  Aire de jeux accessible à tous au jardin du Pa...     14


In [58]:
case_3_target_project_id = 151

for index, row in case_3_district_winning_projects.iterrows():
    result = compute_voter_overlap(case_3_target_project_id, row['project_id'], electionDetails)
    print(f"target project->{case_3_target_project_id} | project_compared_to->{row['project_id']} | overlapping_score_to_target->{result['overlap_percentage_target_project']}")
    print("Overlapping voters")
    print(result['overlapping_voters'],"\n")

target project->151 | project_compared_to->149 | overlapping_score_to_target->0.061224489795918366
Overlapping voters
[4896, 5101, 4913, 1782, 4759, 5017] 

target project->151 | project_compared_to->156 | overlapping_score_to_target->0.6632653061224489
Overlapping voters
[4491, 1037, 4750, 1805, 2576, 1681, 18, 5008, 1301, 2711, 1434, 2716, 2720, 2721, 3874, 4900, 1063, 4906, 813, 1327, 1842, 4147, 3636, 2742, 1591, 4406, 2617, 1335, 2871, 5050, 1597, 2749, 4028, 2499, 2756, 4422, 3912, 715, 77, 2893, 3534, 4049, 5075, 1493, 2902, 2776, 2777, 4186, 5083, 3934, 3170, 4195, 4837, 615, 1129, 3309, 5101, 2416, 3697, 4594, 4979, 2804, 2807, 2043, 893] 

target project->151 | project_compared_to->155 | overlapping_score_to_target->0.061224489795918366
Overlapping voters
[165, 4841, 3309, 2871, 4889, 4186] 

target project->151 | project_compared_to->157 | overlapping_score_to_target->0.19387755102040816
Overlapping voters
[1681, 4759, 3352, 2716, 1951, 2721, 3874, 4900, 813, 4913, 2756, 442